In [1]:
# Make sure in line with LEAP
import mne
import os
import os.path as op
import numpy as np
from os import makedirs
from preprocessor.sourcemodeling import Sourcemodels
from preprocessor.filehandling import get_files
import time
import math

# Temp note: to get source modeled data, the code needs to be executed insequence, as the ncm, fwd solution, and inv
# are computed and stored in the class sourcemodel.

subjects_dir     = '/data/sheng/MEG/'
Subj             = 'case_0452'

ncm_dir          = '/data/sheng/MEG/' + Subj + '/ncm/'
#transMat         = op.join(subjects_dir, Subj + '/freesurfer/coreg/' + Subj + '_' + setName +'_coreg_trans.fif')
coregPath, coregFile = get_files(op.join(subjects_dir, Subj, 'freesurfer/coreg/'), incl_strings = ['trans'],  ftype='.fif')
transMat         = coregPath[0]
fwd_dir          = '/data/sheng/MEG/' + Subj + '/source/ForwardOperators/'
inv_dir          = '/data/sheng/MEG/' + Subj + '/inv/'

# copy below file to variable 'fifFile' in the next block
filepath, filename = get_files(op.join(subjects_dir, Subj, ''), incl_strings = ['spont', 'ica_cleaned'],  ftype='.fif')

print('\n\nselect a *ica_clean.fif file to be processed!\n********************************************************************************** ')
for i, f in enumerate(filename):
    print(i, ': ', f)
print('**********************************************************************************\n\n\n')



select a *ica_clean.fif file to be processed!
********************************************************************************** 
0 :  beers_wesley_spont_rsa_1_chpi_raw_tsss_filt_ica_cleaned.fif
1 :  beers_wesley_spont_rsa_2_chpi_raw_tsss_filt_ica_cleaned.fif
2 :  beers_wesley_spont_rsa_3_chpi_raw_tsss_filt_ica_cleaned.fif
**********************************************************************************





In [18]:
#select with the 'cleaned' file to process, # hard code set name here
FILE_INDEX       = 0
setName          = 'set1'
set_no           = int(setName[3:])

## parcel time series write out
parc             = 'parc2018yeo17_200'
fid_dir          = subjects_dir + Subj + '/fidelity/'
pts_dir          = '/data/sheng/MEG_source_data/'+ Subj + '/'+setName + '/parcel_ts/' 
filepath_where_to_save  = '/data/sheng/MEG_source_data/'+ Subj + '/'+setName + '/parcel_ts_test/' 


######################################################################
# set the sourcemodels class
fifFile          = filepath[FILE_INDEX]  # preprocessed fif file of the spont data
data_fif         = op.join(subjects_dir, Subj, fifFile)
fif_data         = mne.io.read_raw_fif(data_fif, preload=True)
#sourcemodels     = Sourcemodels(fif_data, 'sourcemodel', Subj, subjects_dir = subjects_dir, cor = transMat) 
sourcemodels     = Sourcemodels(fif_data, setName, Subj, subjects_dir = subjects_dir, cor = transMat) 

## check Preprocessor document, it by default writes a file named 'sourcemodel_fwd.fif' to fwd_dir
fwd_file         = fwd_dir + Subj + '_set' + '{:02}'.format(set_no) + '_py-fwd.fif'   

Opening raw data file /data/sheng/MEG/case_0323/molinamireles_daniel_spont_rsa_6_chpi_raw_tsss_filt_ica_cleaned.fif...
    Range : 14000 ... 615999 =     14.000 ...   615.999 secs
Ready.
Reading 0 ... 601999  =      0.000 ...   601.999 secs...


/tmp/ipykernel_1740947/831753186.py:17: RuntimeWarning: This filename (/data/sheng/MEG/case_0323/molinamireles_daniel_spont_rsa_6_chpi_raw_tsss_filt_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  fif_data         = mne.io.read_raw_fif(data_fif, preload=True)


In [19]:
#############################################
# Noise covariance matrices (NCM)           #
#############################################
t1 = time.time()
makedirs(ncm_dir, exist_ok=True)
sourcemodels.make_cov_continous(cov_dir=ncm_dir,n_jobs=12, l_freq=151, h_freq=249, use_empty_room=False)

#############################################
# make forward model                        #
#############################################
makedirs(fwd_dir, exist_ok=True)
#sourcemodels.make_fwd(fwd_dir, spacing=5, conductivity=[0.3], eeg=False, meg=True) # conductivity=[0.3, 0.006, 0.3]
sourcemodels.make_fwd(fwd_dir, spacing=5, eeg=False, meg=True) 


#############################################
#### make inverse model                     #
#############################################
makedirs(inv_dir, exist_ok=True)
sourcemodels.make_inv(inv_dir)


### fidelity weighting ####################################
makedirs(pts_dir, exist_ok=True)
makedirs(fid_dir, exist_ok=True)

# fidelity-weight and save the parcel timeseries as numpy array
sourcemodels.make_fidelity_weighting(pts_dir, fid_dir, parc, eeg=False, meg=True)


print(transMat)
print(inv_dir)
print(fwd_file)
print(pts_dir)

t2 = time.time()
lapse = round( (t2-t1)/ 60, 1)
print('Time spent: ' + str(lapse) + 'min.')


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1.5e+02 - 2.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 151.00
- Lower transition bandwidth: 37.75 Hz (-6 dB cutoff frequency: 132.12 Hz)
- Upper passband edge: 249.00 Hz
- Upper transition bandwidth: 62.25 Hz (-6 dB cutoff frequency: 280.12 Hz)
- Filter length: 89 samples (0.089 s)



[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   6 tasks      | elapsed:    1.2s
[Parallel(n_jobs=12)]: Done  72 tasks      | elapsed:    2.0s
[Parallel(n_jobs=12)]: Done 252 tasks      | elapsed:    3.8s
[Parallel(n_jobs=12)]: Done 329 out of 329 | elapsed:    4.6s finished


Using up to 2959 segments
Number of samples used : 591800
[done]
Computing rank from covariance with rank=None
    Using tolerance 3.5e-15 (2.2e-16 eps * 102 dim * 0.16  max singular value)
    Estimated rank (mag): 73
    MAG: rank 73 computed from 102 data channels with 0 projectors
Computing rank from covariance with rank=None
    Using tolerance 3.9e-14 (2.2e-16 eps * 204 dim * 0.86  max singular value)
    Estimated rank (grad): 71
    GRAD: rank 71 computed from 204 data channels with 0 projectors
Computing rank from covariance with rank=None
    Using tolerance 2.6e-15 (2.2e-16 eps * 23 dim * 0.51  max singular value)
    Estimated rank (eeg): 23
    EEG: rank 23 computed from 23 data channels with 0 projectors


/home/sheng/anaconda3/envs/mne/lib/python3.12/site-packages/preprocessor/sourcemodeling.py:440: RuntimeWarning: Something went wrong in the data-driven estimation of the data rank as it exceeds the theoretical rank from the info (73 > 71). Consider setting rank to "auto" or setting it explicitly as an integer.
  fig_cov, fig_svd = cov.plot(data.info, show=False)


Setting up the source space with the following parameters:

SUBJECTS_DIR = /data/sheng/MEG
Subject      = case_0323
Surface      = white
Approximate spacing 5 mm

>>> 1. Creating the source space...

Loading /data/sheng/MEG/case_0323/surf/lh.white...
Mapping lh case_0323 -> spacing (5) ...
    Triangle neighbors and vertex normals...
    Vertex neighbors...
    Decimating...
loaded lh.white 4849/112573 selected to source space (spacing = 5)

Loading /data/sheng/MEG/case_0323/surf/rh.white...
Mapping rh case_0323 -> spacing (5) ...
    Triangle neighbors and vertex normals...
    Vertex neighbors...
    Decimating...
loaded rh.white 4796/112656 selected to source space (spacing = 5)

Calculating source space distances (limit=inf mm)...


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   36.0s remaining:   59.9s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   36.8s remaining:   22.1s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   37.2s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   3 out of   8 | elapsed:   34.2s remaining:   57.1s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:   34.6s remaining:   20.8s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:   35.1s finished


    Computing patch statistics...
    Patch information added...
    Computing patch statistics...
    Patch information added...
You are now one step closer to computing the gain matrix
Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
outer skin  CM is  -0.24 -24.48   1.81 mm
outer skull CM is  -0.24 -24.25   1.39 mm
inner skull CM is  -0.29 -28.78   9.15 mm
Checking that surface outer skull is inside surface outer skin  ...
Checking that surface inner skull is inside surface outer skull ...
Checking distance between outer skin  and outer skull surfaces...
Minimum distance between the outer skin  and outer skull surfaces is approximately    1.4 mm
Checking distance between outer skull and inner skull surfaces...
Minimum distance between the outer skull and inner skull surfaces is app